In [1]:
!bash download_voc.sh

--2021-02-28 11:11:11--  http://www.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/semantic_contours/benchmark.tgz
Resolving www.eecs.berkeley.edu (www.eecs.berkeley.edu)... 23.185.0.1, 2620:12a:8001::1, 2620:12a:8000::1
Connecting to www.eecs.berkeley.edu (www.eecs.berkeley.edu)|23.185.0.1|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/semantic_contours/benchmark.tgz [following]
--2021-02-28 11:11:11--  https://www.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/semantic_contours/benchmark.tgz
Connecting to www.eecs.berkeley.edu (www.eecs.berkeley.edu)|23.185.0.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www2.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/semantic_contours/benchmark.tgz [following]
--2021-02-28 11:11:11--  https://www2.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/semantic

In [2]:
!wget 'https://raw.githubusercontent.com/bonlime/keras-deeplab-v3-plus/master/extract_weights.py'
!wget 'https://raw.githubusercontent.com/bonlime/keras-deeplab-v3-plus/master/load_weights.py'
!wget 'https://raw.githubusercontent.com/bonlime/keras-deeplab-v3-plus/master/model.py'

--2021-02-28 11:18:24--  https://raw.githubusercontent.com/bonlime/keras-deeplab-v3-plus/master/extract_weights.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3648 (3.6K) [text/plain]
Saving to: ‘extract_weights.py’

extract_weights.py  100%[===================>]   3.56K  --.-KB/s    in 0s      

2021-02-28 11:18:25 (59.0 MB/s) - ‘extract_weights.py’ saved [3648/3648]

--2021-02-28 11:18:25--  https://raw.githubusercontent.com/bonlime/keras-deeplab-v3-plus/master/load_weights.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 

In [1]:
from VOCGen import get_voc_datagen
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LambdaCallback
from tensorflow.keras.losses import categorical_crossentropy
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import binary_crossentropy, BinaryCrossentropy
import os
from metrics import mean_iou
from gan_model_semi import AdverGAN
from tensorflow.keras.metrics import Mean, BinaryAccuracy
from tensorflow.keras.losses import categorical_crossentropy
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [2]:
N_CLASS = 20 + 1 
BATCH_SIZE = 8
preprocess_input = lambda x : x / 255.
target_size = (320, 320)
tr_gen, val_gen = get_voc_datagen(N_CLASS, BATCH_SIZE, preprocess_input, None, target_size, 0.25)

In [3]:
def masked_categorical_crossentropy(y_pred, conf, thres, n_class):
  pse_true_labels = tf.argmax(y_pred, axis=-1)
  pse_true = tf.one_hot(pse_true_labels, depth=n_class)

  loss_mask = tf.where(conf > thres, 1, 0)
  loss_mask_float = tf.cast(loss_mask, tf.float32)
  normal_loss =  categorical_crossentropy(pse_true, y_pred)
  expanded_normal = tf.expand_dims(normal_loss, axis=-1)

  masked_loss = loss_mask_float * expanded_normal
  return masked_loss

In [4]:
g_iou_tracker = Mean(name='iou')
gan_acc_tracker = Mean(name='gan_acc')
disc_acc_tracker = Mean(name='disc_acc')
iou_score = mean_iou
binary_entropy = BinaryCrossentropy()
gan_acc = BinaryAccuracy()
disc_acc = BinaryAccuracy()

In [5]:
# Lambda adv -> labeled -> 0.01 unlabeled -> 0.001
# Lambda semi -> 0.1 thres 0.2
gan_model = AdverGAN(1e-2, 0.1, 0.2, target_size, N_CLASS)
g_opt = Adam(2.5e-4)
d_opt = Adam(1e-5)

In [6]:
BEST_VAL_IOU = 0.0
MODEL_SAVE_DIR = os.path.join('drive', 'MyDrive', 'adverseg_models')

def save_generator_discriminator_on_best(logs):
  global BEST_VAL_IOU
  if BEST_VAL_IOU < logs['val_gen_iou']:
    BEST_VAL_IOU = logs['val_gen_iou']
    gan_model.save_models(MODEL_SAVE_DIR)


saving_model_callback = LambdaCallback(
    on_epoch_end=lambda epoch, logs: save_generator_discriminator_on_best(logs)
)

In [7]:
gan_model.compile(g_opt, d_opt, binary_entropy, gan_acc, disc_acc,
                  categorical_crossentropy, iou_score, masked_categorical_crossentropy, 
                  g_iou_tracker, gan_acc_tracker, disc_acc_tracker)

In [8]:
gan_model.fit(tr_gen, epochs=40, validation_data=val_gen, callbacks=[saving_model_callback])

Epoch 1/40
  1/797 [..............................] - ETA: 18:34:58 - disc_fake_loss: 0.7151 - disc_real_loss: 0.6822 - gan_loss: 0.2165 - gen_iou: 0.7406 - g_loss: 0.2037 - disc_acc: 0.2785 - gan_acc: 0.9078

KeyboardInterrupt: ignored